In [2]:
import numpy as np

# network goes R50 --> R20 --> R20 --> R10
w_1 = np.random.normal(0,0.05,(50, 20))
b_1 = np.random.normal(0,0.05,20)

w_2 = np.random.normal(0,0.05,(20, 10))
b_2 = np.random.normal(0,0.05,10)

w_3 = np.random.normal(0,0.05,(10, 1))
b_3 = np.random.normal(0,0.05,1)

X = np.random.normal(0, 1, (10000, 50))
y = np.array([np.sum(X[x,:]**2) for x in range(10000)])

X_train = X[:8000, :]
X_test = X[8000:, :]

y_train = np.sum(X[:8000, :], axis=1)
y_test = np.sum(X[8000:, :], axis=1)

def loss(y, y_pred):
    return np.sum(.5*(y-y_pred)**2, axis=1)

def dloss(y, y_pred):
    return np.sum(y-y_pred, axis=1)

def activation(z):
    return z * (z>0)

def dactivation(z):
    return 1 * (z>0)

test_losses = []

def forward(X, params):
    w_1, b_1, w_2, b_2, w_3, b_3 = params
    z_1 = X @ w_1 + b_1
    a_1 = activation(z_1)

    z_2 = a_1 @ w_2 + b_2
    a_2 = activation(z_2)

    z_3 = a_2 @ w_3 + b_3
    a_3 = activation(z_3)

    return a_3, a_2, a_1, z_3, z_2, z_1


In [7]:
epochs = 1
for _ in range(epochs):    
    a_3, a_2, a_1, z_3, z_2, z_1 = forward(X_train, (w_1, b_1, w_2, b_2, w_3, b_3))
    out_test, _, _, _, _, _ = forward(X_test, (w_1, b_1, w_2, b_2, w_3, b_3))
    l = loss(y_train, a_3)
    dl_da_3 = dloss(y_train, a_3) # 8000 x 1
    
    da_3_dz_3 = dactivation(z_3) # 8000 x 1
    dz_3_dw_3 = a_2 # 8000 x 10 x 1; to get z_3 we do a_2 (8000x10) @ w_3 (10 x 1) result is (8000x1)
    dz_3_db_3 = np.ones((z_3.shape)) # 8000 x 1

    dl_dw_3 = dl_da_3 * da_3_dz_3 * dz_3_dw_3
    dl_dwb_3 = dl_da_3 * da_3_dz_3 * dz_3_db_3

    da_2_dz_2 = 



    test_losses.append(np.mean(loss(y_test, out_test)))

In [ ]:
w_1 = np.random.normal(0,0.05,(3,2))
b_1 = np.random.normal(0,0.05,3)

w_2 = np.random.normal(0,0.05,(2,3))
b_2 = np.random.normal(0,0.05,2)

X = np.random.normal(0, 1, (10000, 2))
y = np.array([np.sum(X[x,:]**2) for x in range(10000)])

X_train = X[:8000, :]
X_test = X[8000:, :]

y_train = np.sum(X[:8000, :], axis=1)
y_test = np.sum(X[8000:, :], axis=1)

epochs = 10

def loss(y, y_pred):
    return np.sum(.5*(y-y_pred)**2)

def dloss(y, y_pred):
    return np.sum(y-y_pred)

def activation(z):
    return z * (z>0)

def dactivation(z):
    return 1 * (z>0)

test_losses = []

def forward(X, params):
    w_1, b_1, w_2, b_2 = params
    z_1 = X @ w_1 + b_1
    a_1 = activation(z_1)

    z_2 = a_1 @ w_2 + b_2
    a_2 = activation(z_2)

    return a_2, a_1, z_2, z_1

for _ in range(epochs):
    for i in range(8000):
        # dC/W = dC/da * da/dz * dz/dW
        
        # Compute dC/da(L) and pass this to the last layers backward
        # loop:
        # 1) compute da(l)/dz(l) and matrix multiply with the passed param
        # 2) compute dz(l)/dW(l) where W is the flattened gradient matrix
        # 3) matrix multiply 2) with 1) to get the weight gradient
        # 4) compute dz(l)/db(l) and multiply with 1) to get the bias gradient
        # 5) apply weight and bias gradients
        # 6) compute dz(l)/da(l-1) and matrix multiply with 1)
        # 7) return 6) to be passed to the next layer

        # dC/da(L) * da(L)/dz(L) 
        # z = Wa(l-1) + b
        # a(l-1) = f(W(l-1)a(l-2) + b(l-2))
        # dC/dW(L-1) = dC/da(L) * da(L)/dz(L) * dz(l)/da(l-1) * da(l-1)/dz(l-1) * dz(l-1)/dW(l-1)
        
        a_2, a_1, z_2, z_1 = forward(X_train, (w_1, b_1, w_2, b_2))
        out_test, _, _, _, = forward(X_test, (w_1, b_1, w_2, b_2))
        l = loss(y_train, a_2)
        dl_da_2 = dloss(y_train, a_2) # dC/da

        da_2_dz_2 = dactivation(z_2,) # da/dz
        dz_2_dw_2 = a_2
        dz_2_db_2 = np.ones((z_2.shape))

        weight_factor_1 = dl_da_2 @ da_2_dz_2 @ dz_2_dw_2
        bias_factor_1 = dl_da_2 @ da_2_dz_2 @ dz_2_db_2

        

        